## Notebook to plot example reports 

plot prediction, saliencies and MRI 

need to have run pipeline beforehand to get back prediction in native space


In [31]:
import os
import sys
import numpy as np
import h5py
import matplotlib_surface_plotting as msp
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import nibabel as nb
import meld_graph.experiment
from meld_classifier.paths import BASE_PATH
from meld_classifier.meld_cohort import MeldCohort,MeldSubject


In [32]:
def load_prediction(subject,hdf5,dset='prediction_clustered'):
    results={}
    with h5py.File(hdf5, "r") as f:
        for hemi in ['lh','rh']:
            results[hemi] = f[subject][hemi][dset][:]
    return results


### plot salient vertices and saliencies 

In [33]:
from meld_graph.dataset import GraphDataset
from meld_graph.evaluation import Evaluator
import matplotlib as mpl
import matplotlib.cm as cm
from meld_graph.data_preprocessing import Preprocess
from meld_graph.confidence import get_confidence
import matplotlib_surface_plotting as msp
from meld_classifier.meld_plotting import trim
from PIL import Image


In [34]:
def get_subj_data(subject_id, eva):
    
    #load data for that subject
    data_dictionary = eva.load_data_from_file(subject_id, keys=['result','cluster_thresholded','input_features'], 
                                          split_hemis=True, )
    features_vals = data_dictionary['input_features']
    predictions = data_dictionary['cluster_thresholded']

    #find thresholds used if two thresholds
    if isinstance(eva.threshold, np.ndarray):
        if max(data_dictionary['result']['left'].max(), data_dictionary['result']['right'].max()) > eva.threshold[1]:
            threshold_text = "high confidence cluster"
        else :
            threshold_text = "No high confidence cluster\nLow confidence cluster given instead"
    else:
        threshold_text = ""
    
    #find clusters and load saliencies and confidence
    list_clust = {}
    confidences = {}
    saliencies = {}
    for hemi in ['left','right']:
        list_clust[hemi] = set(predictions[hemi])
        list_clust[hemi].remove(0.0)
        keys = [f'saliencies_{cl}' for cl in list_clust[hemi]] + [f'mask_salient_{cl}' for cl in list_clust[hemi]]
        saliencies.update(eva.load_data_from_file(subject_id, 
                                            keys=keys, 
                                            split_hemis=True))
    
        for cl in list_clust[hemi]:
            mask_salient = saliencies[f'mask_salient_{cl}'][hemi].astype(bool)
            confidence_cl_salient = np.max(data_dictionary['result'][hemi][mask_salient])
            confidences[f'confidence_{cl}'] =  confidence_cl_salient

    return list_clust, features_vals, predictions, threshold_text, saliencies, confidences

def load_cmap():
    """ create the colors dictionarry for the clusters"""
    from matplotlib.colors import ListedColormap
    import numpy as np
    colors =  [
        [255,0,0],     #red
        [255,215,0],   #gold
        [0,0,255],     #blue
        [0,128,0],     #green
    ]
    colors=np.array(colors)/255
    dict_c = dict(zip(np.arange(1, len(colors)+1), colors))
    cmap = ListedColormap(colors)
    return cmap, dict_c

def create_surface_plots(surf,prediction,c, base_size=20):
    """plot and reload surface images"""
    cmap, colors =  load_cmap()
    tmp_file = 'tmp.png'
    msp.plot_surf(surf['coords'],
              surf['faces'],prediction,
              rotate=[90],
              mask=prediction==0,pvals=np.ones_like(c.cortex_mask),
              colorbar=False,vmin=1,vmax=len(colors) ,cmap=cmap,
              base_size=base_size,
              filename=tmp_file)
    im = Image.open(tmp_file)
    im = trim(im)
    im = im.convert("RGBA")
    im1 = np.array(im)
    msp.plot_surf(surf['coords'],
            surf['faces'],prediction,
              rotate=[270],
              mask=prediction==0,pvals=np.ones_like(c.cortex_mask),
              colorbar=False,vmin=1,vmax=len(colors),cmap=cmap,
              base_size=base_size,
              filename=tmp_file)
    im = Image.open(tmp_file)
    im = trim(im)
    im = im.convert("RGBA")
    im2 = np.array(im)
    plt.close('all')
    os.remove(tmp_file)
    return im1,im2

def get_key(dic, val):
    # function to return key for any value in dictionnary
    for key, value in dic.items():
        if val == value:
            return key
    return "No key for value {}".format(val)

def define_atlas():
    file = os.path.join(BASE_PATH, "fsaverage_sym", "label", "lh.aparc.annot")
    atlas = nb.freesurfer.io.read_annot(file)
    vertex_i = np.array(atlas[0]) - 1000  # subtract 1000 to line up vertex
    rois_prop = [
        np.count_nonzero(vertex_i == x) for x in set(vertex_i)
    ]  # proportion of vertex per rois
    rois = [x.decode("utf8") for x in atlas[2]]  # extract rois label from the atlas
    rois = dict(zip(rois, range(len(rois))))  # extract rois label from the atlas
    rois.pop("unknown")  # roi not part of the cortex
    rois.pop("corpuscallosum")  # roi not part of the cortex
    return rois, vertex_i, rois_prop

def get_cluster_location(cluster_array):
    cluster_array = np.array(cluster_array)
    rois, vertex_i, rois_prop = define_atlas()
    pred_rois = list(vertex_i[cluster_array])
    pred_rois = np.array([[x, pred_rois.count(x)] for x in set(pred_rois) if x != 0])
    ind = pred_rois[np.where(pred_rois == pred_rois[:,1].max())[0]][0][0]
    location = get_key(rois,ind)
    return location



In [35]:
dataset = 'H101'  # ""test" or "H101"

# subjects=[
#         'MELD_H16_3T_FCD_004',
#           # 'MELD_H4_3T_FCD_0011'
#         ]

subjects=[
        #'MELD_H101_3T_FCD_00068' # low confidence one , not used anymore removed
        
        #  'MELD_H101_3T_FCD_00138', # high confidence, MRI+ve H101
        #  'MELD_H101_3T_FCD_00062', #  low confidence, MRI-ve H101

         'MELD_H101_3T_FCD_00108', #  high confidence, MRI-ve H101
         'MELD_H101_3T_FCD_00121', #  low confidence, MRI-ve H101
          ]

In [ ]:
# load experiment
model_graph = 'experiments_graph/kw350/23-10-30_LVHZ_dcp/s_0/fold_all_newthreshold'
exp = meld_graph.experiment.Experiment.from_folder(model_graph)
exp.data_parameters["augment_data"] = {}

#load trainval dataset
split = "test"

# if load subjects from test 
if dataset == 'H101':
    save_dir = 'experiments_graph/kw350/23-10-30_LVHZ_dcp/s_0/fold_all_newthreshold/test_H27H28H101'
    cohort = MeldCohort(
            hdf5_file_root="{site_code}_{group}_featurematrix_combat_freesurfer_harmonised_NewSite.hdf5",
            dataset='MELD_dataset_NewSiteH27H28H101_freesurfer.csv',
    )
else:
    save_dir=None
    cohort = MeldCohort(
        hdf5_file_root=exp.data_parameters["hdf5_file_root"],
        dataset=exp.data_parameters["dataset"],
    )


features = exp.data_parameters["features"]

dataset = GraphDataset(subjects, cohort, exp.data_parameters, mode="test")
save_prediction_suffix=""

# create evaluator
eva = Evaluator(
    experiment=exp,
    checkpoint_path=model_graph,
    make_images=True,
    dataset=dataset,
    save_dir=save_dir,
    cohort=cohort,
    subject_ids=subjects,
    mode="test",
    thresh_and_clust=True,
    threshold='slope_threshold',
)


In [11]:
# # plot prediction and lesion
# eva.plot_subjects_prediction()

In [12]:
# # calculate saliencies if does not exists
# eva.calculate_saliency(save_prediction_suffix="")

In [37]:
input_dir = 'meld_data/Example_pts_graph/input'
output_dir = 'meld_data/Example_pts_graph/output'
 

In [38]:

 # setup parameters
base_feature_sets = [
    ".on_lh.gm_FLAIR_0.5.sm3.mgh",
    ".on_lh.wm_FLAIR_1.sm3.mgh",
    ".on_lh.curv.sm3.mgh",
    ".on_lh.pial.K_filtered.sm20.mgh",
    ".on_lh.sulc.sm3.mgh",
    ".on_lh.thickness_regression.sm3.mgh",
    ".on_lh.w-g.pct.sm3.mgh",
]

feature_names_sets = [
    "GM FLAIR (50%)",
    "WM FLAIR (1mm)",
    "Mean curvature",
    "Intrinsic Curvature",
    "Sulcal depth",
    "Cortical thickness",
    "Grey-white contrast",
]
NVERT=293804

for subject_id in subjects:
    subject = MeldSubject(subject_id, cohort=cohort)

    #create results folder
    os.makedirs(os.path.join(output_dir,subject_id,'reports'), exist_ok=True)
    # initialise parameter for plot
    fig = plt.figure(figsize=(15, 8), constrained_layout=True)
    if subject.has_flair:
        base_features = base_feature_sets
        feature_names = feature_names_sets
    else:
        base_features = base_feature_sets[2:]
        feature_names = feature_names_sets[2:]
    # load predictions and data subject
    list_clust, features_vals, predictions, threshold_text, saliencies, confidences  = get_subj_data(subject_id, eva)
    # Loop over hemi
    for i, hemi in enumerate(["left", "right"]):
        # prepare grid plot
        gs1 = GridSpec(2, 3, width_ratios=[1, 1, 1], wspace=0.1, hspace=0.1)
        gs2 = GridSpec(2, 4, height_ratios=[1, 3], width_ratios=[1, 1, 0.5, 2], wspace=0.1)
        gs3 = GridSpec(1, 1)
        # plot predictions on inflated brain
        im1, im2 = create_surface_plots(cohort.surf, prediction=predictions[hemi], c=cohort)
        if hemi == "right":
            im1 = im1[:, ::-1]
            im2 = im2[:, ::-1]
        ax = fig.add_subplot(gs1[i, 1])
        ax.imshow(im1)
        ax.axis("off")
        title = 'Left hemisphere' if hemi=='left' else 'Right hemisphere'
        ax.set_title(title, loc="left", fontsize=20)
        ax = fig.add_subplot(gs1[i, 2])
        ax.imshow(im2)
        ax.axis("off")
        # initiate params for saliencies
        prefixes = [".combat", ".inter_z.intra_z.combat", ".inter_z.asym.intra_z.combat"]
        cmap = mpl.colors.LinearSegmentedColormap.from_list(
            "grpr",
            colors=[
                "#276419",
                "#FFFFFF",
                "#8E0152",
            ],
        )
        labels = ["Harmonised", "Normalised", "Asymmetry"]
        hatching = ["\\\\", "//", "--"]
        # loop over clusters
        for cluster in list_clust[hemi]:
            fig2 = plt.figure(figsize=(17, 9))
            # get and plot saliencies
            saliencies_cl = saliencies[f'saliencies_{cluster}'][hemi]
            saliencies_cl = saliencies_cl * (NVERT/2)
            # plot prediction and salient vertices
            mask = np.array([predictions[hemi] == cluster])[0]
            mask_salient = saliencies[f'mask_salient_{cluster}'][hemi].astype(bool)
            mask_comb = mask.astype(int)+mask_salient.astype(int)                
            lims_saliencies_cl = 1.1*np.max([np.max(np.mean(saliencies_cl[mask_salient], axis=0)),-np.min(np.mean(saliencies_cl[mask_salient], axis=0))])
            norm = mpl.colors.Normalize(vmin=-lims_saliencies_cl, vmax=lims_saliencies_cl)
            m = cm.ScalarMappable(norm=norm, cmap=cmap)
            
            im1, im2 = create_surface_plots(cohort.surf, prediction=mask_comb, c=cohort, base_size=10)
            if hemi == "right":
                im1 = im1[:, ::-1]
                im2 = im2[:, ::-1]
            ax2 = fig2.add_subplot(gs2[1, 0])
            ax2.imshow(im1)
            ax2.axis("off")                
            ax2 = fig2.add_subplot(gs2[1, 1])
            ax2.imshow(im2)
            ax2.axis("off")
            # ax2.set_title('Predicted cluster and\n20% most salient vertices', loc="left", fontsize=15)
            ax2 = fig2.add_subplot(gs2[:, 3],)
            for pr, prefix in enumerate(prefixes):
                cur_data = np.zeros(len(base_features))
                cur_err = np.zeros(len(base_features))
                saliency_data = np.zeros(len(base_features))
                for b, bf in enumerate(base_features):
                    cur_data[b] = np.mean(
                        np.array(features_vals[hemi][mask_salient, features.index(prefix + bf)])
                    )
                    cur_err[b] = np.std(
                        np.array(features_vals[hemi][mask_salient, features.index(prefix + bf)])
                    )
                    saliency_data[b] = np.mean(
                        saliencies_cl[mask_salient ,features.index(prefix + bf)]
                        )
                ax2.barh(
                    y=np.array(range(len(base_features))) - pr * 0.3,
                    width=cur_data,
                    hatch=hatching[pr],
                    height=0.3,
                    edgecolor="k",
                    xerr=cur_err,
                    label=labels[pr],
                    color=m.to_rgba(saliency_data),
                )
            limvals = np.max([np.max(cur_data+cur_err),-np.min(cur_data-cur_err)])+0.5
            ax2.set_xlim([-limvals, limvals])
            # ax2.set_xticks([])
            ax2.set_yticks(np.array(range(len(base_features))) - 0.23)
            ax2.set_yticklabels(feature_names, fontsize=16)
            ax2.set_xlabel("Z score", fontsize=16)
            ax2.legend(loc="upper center", bbox_to_anchor=(0.5, 1.17), fontsize=16)
            fig2.colorbar(m, ax=ax2).set_label(label='Saliency',size=18,weight='bold')
            ax2.set_autoscale_on(True)
            ## display info cluster
            # get size
            size_clust = np.sum(cohort.surf_area[predictions[hemi] == cluster]) / 100
            size_clust = round(size_clust, 3)
            # get location
            location = get_cluster_location(predictions[hemi] == cluster)
            # get confidence
            confidence = round(confidences[f'confidence_{cluster}'].mean(),2)
            # plot info in text box in upper left in axes coords
            textstr = "\n".join(
                (
                    f" Cluster {int(cluster)} on the {hemi} hemisphere",
                    " ",
                    f" Cluster size = {size_clust} cm2",
                    " ",
                    f" Cortical region =  {location}",
                        " ",
                    f" Confidence score =  {confidence}",
                    " ",
                    f"-{threshold_text}",
                    
                    
                )
            )
            props = dict(boxstyle="round", alpha=0.5)
            ax2 = fig2.add_subplot(gs2[0, 0:2])
            ax2.text(0.05, 0.95, textstr, transform=ax2.transAxes, fontsize=18, verticalalignment="top", bbox=props)
            ax2.axis("off")
            fig2.savefig(f"{output_dir}/{subject_id}/reports/saliency_{subject.subject_id}_{hemi}_c{int(cluster)}_combat.png", facecolor='white')

Loading and preprocessing test data


MELD_H101_3T_FCD_00108


Z-scoring data for MELD_H101_3T_FCD_00108
Loading and preprocessing test data


MELD_H101_3T_FCD_00121


Z-scoring data for MELD_H101_3T_FCD_00121


In [40]:
### plot MRI
import glob
from nilearn import plotting, image
from nilearn.image import new_img_like
from nilearn._utils.numpy_conversions import as_ndarray

subject_id= 'MELD_H101_3T_FCD_00121'

# Open their MRI data if available
t1_file = glob.glob(os.path.join(input_dir,subject_id,'T1', '*.nii.gz'))[0]
prediction_file = glob.glob(os.path.join(output_dir, subject_id, "predictions", "prediction*"))[0]
# load image
imgs = {
    "anat": nb.load(t1_file),
    "pred": nb.load(prediction_file),
}
# # Resample and move to same shape and affine than t1
imgs["pred"] = image.resample_img(
    imgs["pred"],
    target_affine=imgs["anat"].affine,
    target_shape=imgs["anat"].shape,
    interpolation="nearest",
    copy=True,
    order="F",
    clip=False,
    fill_value=0,
    force_resample=False,
)
# initialise parameter for plot
fig = plt.figure(figsize=(15, 8), constrained_layout=True)
# Loop over hemi
for i, hemi in enumerate(["left", "right"]):
    # prepare grid plot
    gs3 = GridSpec(1, 1)
    for cluster in list_clust[hemi]:
        # plot cluster on anat MRI volume
        fig3 = plt.figure(figsize=(15, 8))
        ax3 = fig3.add_subplot(gs3[0])
        min_v = cluster - 1
        max_v = cluster + 1
        mask = image.math_img(f"(img < {max_v}) & (img > {min_v})", img=imgs[f"pred"])
        coords = plotting.find_xyz_cut_coords(mask)
        vmax = np.percentile(imgs["anat"].get_fdata(), 99)
        display = plotting.plot_anat(
                        t1_file, colorbar=False, cut_coords=coords, 
                        draw_cross=False, radiological=True,
                        figure=fig3, axes=ax3, vmax=vmax
                    )        
        for cut_ax in display.axes.values():
                    slices_x = np.linspace(cut_ax.ax.get_xlim()[0], cut_ax.ax.get_xlim()[1],100)
                    cut_ax.ax.set_xlim(slices_x[12], slices_x[-12])
                    slices_y = np.linspace(cut_ax.ax.get_ylim()[0], cut_ax.ax.get_ylim()[1],100)
                    cut_ax.ax.set_ylim(slices_y[12], slices_y[-12])
        fig3.savefig(f"{output_dir}/{subject_id}/reports/mri_{subject_id}_{hemi}_c{int(cluster)}_rawT1.png")     
        #display cluster
        data = imgs["pred"].get_fdata()
        map_img = new_img_like(imgs["pred"], as_ndarray((data==cluster) | (data==cluster*100)).astype(float), imgs["pred"].affine)
        display.add_contours(
                map_img,
                levels=[0.5],
                colors=["red"],
                filled=True,
                alpha=0.7,
                linestyles="solid",
            )
        # display cluster salient vertices
        map_img = new_img_like(imgs["pred"], as_ndarray(data==cluster*100).astype(float), imgs["pred"].affine)
        display.add_contours(
                map_img,
                levels=[0.5],
                colors=["yellow"],
                filled=True,
                alpha=0.7,
                linestyles="solid",
            )
        fig3.savefig(f"{output_dir}/{subject_id}/reports/mri_{subject_id}_{hemi}_c{int(cluster)}_T1pred.png")

        # display FLAIR if exists
        flair_file = glob.glob(os.path.join(input_dir,subject_id,'FLAIR', '*.nii.gz'))[0]
        if os.path.isfile(flair_file):
            fig4 = plt.figure(figsize=(15, 8))
            ax4 = fig4.add_subplot(gs3[0])
            vmax = np.percentile(nb.load(flair_file).get_fdata(), 99)
            display2 = plotting.plot_anat(
                            flair_file, colorbar=False, cut_coords=coords, 
                            draw_cross=False, radiological=True,
                            figure=fig4, axes=ax4, vmax=vmax
                        )
            for cut_ax in display2.axes.values():
                    slices_x = np.linspace(cut_ax.ax.get_xlim()[0], cut_ax.ax.get_xlim()[1],100)
                    cut_ax.ax.set_xlim(slices_x[12], slices_x[-12])
                    slices_y = np.linspace(cut_ax.ax.get_ylim()[0], cut_ax.ax.get_ylim()[1],100)
                    cut_ax.ax.set_ylim(slices_y[12], slices_y[-12])
            fig4.savefig(f"{output_dir}/{subject_id}/reports/mri_{subject_id}_{hemi}_c{int(cluster)}_raw_FLAIR.png")     
        